In [2]:
import json
from pytorch3d.io import load_obj
from transforms import YForward2NegZForward, NegZForward2YForward
from wandb_utils.wandb import WandB
import torch
import numpy as np

# run1 = WandB(
#         project='objdet_GT_Viz',
#         enabled=True,
#         log_objects=True,
#         name='GT Train Viz - Multiview: '
#     )

In [3]:
json_mapping = '/multiview/3d-count/obj_detection/scripts/json_kitti_mapping.json'
f = open(json_mapping)
json_map = json.load(f)
f.close()

In [8]:
json_map['299fa980-f6aa-419f-a996-f7601b45e2bc-0']
# id = '299fa980-f6aa-419f-a996-f7601b45e2bc-0'
# id = '299fa980-f6aa-419f-a996-f7601b45e2bc-2'

{'24k_frustum_label': '/multiview/3d-count/obj_detection/normalized_data/labels/299fa980-f6aa-419f-a996-f7601b45e2bc-0.json',
 '24k_frustum_point_cloud_path': '/multiview/3d-count/obj_detection/normalized_data/obj/299fa980-f6aa-419f-a996-f7601b45e2bc-0.obj',
 'obj_detection_kitti_path': '/multiview/3d-count/obj_detection/normalized_data/training/label_2/000000.txt'}

In [65]:
label_path = json_map[id]["obj_detection_kitti_path"]
f = open(label_path,'r',encoding = 'utf-8')
data = f.readlines()    # read the first 4 data
# f.close()

In [66]:
dims = []
loc = []
shift = NegZForward2YForward()
for i in data:
    centroid = []
    dimensions = []
    dimensions.append(float(i.split()[8]))
    dimensions.append(float(i.split()[9]))
    dimensions.append(float(i.split()[10]))
    # print(dimensions)
    # dims.append(shift(torch.tensor(dimensions)).tolist())
    dims.append(dimensions)
    centroid.append(float(i.split()[11]))
    centroid.append(float(i.split()[12]))
    centroid.append(float(i.split()[13]))
    # loc.append(shift(torch.tensor(centroid)).tolist())
    loc.append(centroid)
dims = np.array(dims)
loc = np.array(loc)


In [67]:
box_list = []
labels = []

boxes = run1.get_bb_dict(centroids=loc, dims=dims, labels=labels)

obj_path = json_map[id]["24k_frustum_point_cloud_path"]
pc, _, _ = load_obj(obj_path)
# pc = shift(pc)
pc_dict = run1.get_point_cloud_log(pc, boxes=boxes)

[[0.64134446 0.20315426 0.20315422]
 [0.64134446 0.20315426 0.20315423]
 [0.64134446 0.20315426 0.20315416]
 [0.64134446 0.20315426 0.20315433]
 [0.64134446 0.20315426 0.20315421]]


In [68]:
run1.log(pc_dict)
run1.finish()